In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import time
from tqdm import tqdm

In [14]:
for i in tqdm(range(301, 395), desc="Scraping pages", position=1, colour="green"):
    url = f"https://animekai.bz/browser?keyword=&type%5B%5D=movie&type%5B%5D=tv&type%5B%5D=ova&type%5B%5D=ona&type%5B%5D=special&sort=title_az&page={i}"
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    animekai_webpage = requests.get(url, headers = headers).text
    soup2 = BeautifulSoup(animekai_webpage, "lxml")
    
    en_titles = [tag.text.strip() for tag in soup2.find_all('a', class_ = "title")]
    jp_titles = [i.get("data-jp", "np.nan").strip() for i in soup2.find_all("a", class_="title")]
    ep_list = []
    type_list = []

    sub_list = [
        tag.find("span", class_="sub").text if tag.find("span", class_="sub") else "NaN" 
        for tag in soup2.find_all("div", class_ = "aitem")
    ]
    dub_list = [
        int(tag.find("span", class_="dub").text) if tag.find("span", class_="dub") and tag.find("span", class_="dub").text.isdigit() else np.nan 
        for tag in soup2.find_all("div", class_ = "aitem")
    ]
    episodes_and_type = [
        [span.find("b").text.strip() for span in tag.find_all("span") if span.find("b")]
        for tag in soup2.find_all("div", class_="aitem")
    ]
    
    for entry in episodes_and_type:
        if len(entry) == 2:
            ep_val = int(entry[0]) if entry[0].isdigit() else np.nan
            type_val = entry[1]
        elif len(entry) == 1:
            ep_val = np.nan
            type_val = entry[0]
        else:
            ep_val = np.nan
            type_val = np.nan
    
        ep_list.append(ep_val)
        type_list.append(type_val)
    tag_list = []
    url_list = []
    genres_list = []
    country_list = []
    premiered_list = []
    date_aired_list = []
    broadcast_list = []
    duration_list = []
    status_list = []
    rating_list = []
    votes_list = []
    studios_list = []
    producers_list = []
    for tag in soup2.find_all("div", class_ = "tags"):
        if tag.find(class_ = "adult"):
            tag_list.append(True)
        else:
            tag_list.append(False)
    page_urls = [block.find("a", class_="poster").get("href") for block in soup2.find_all("div", class_="aitem")]
    
    for page_url in tqdm(page_urls, desc=f"Details on page {i}", colour="blue", leave=False, position=0):
        time.sleep(1)
        link = "https://animekai.bz" + page_url
        anime = requests.get(link, headers = headers).text
        soup = BeautifulSoup(anime, "lxml")
        url_list.append(link)
            
        detail_block = soup.find("div", class_="detail")
        if not detail_block:
            continue
        genres, country, premiered, date_aired, broadcast, episodes, duration, status, rating, votes, studios, producers = [],"","","","",np.nan,"","",np.nan,"","","" 
        for div in detail_block.find_all("div"):
            text = div.text.strip()
            span = div.find("span")
            if text.startswith("Genres:"):
                genres = [a.text.strip() for a in span.find_all("a")] if span else "NaN"
            elif text.startswith("Country:"):
                country = span.find("a").text.strip() if span and span.find("a") else "NaN"
            elif text.startswith("Premiered:"):
                premiered = div.find("span").text.strip() if span else "NaN"
            elif text.startswith("Date aired:"):
                date_aired = div.find("span").text.strip() if span else "NaN"
            elif text.startswith("Broadcast:"):
                broadcast = div.find("span").text.strip() if span else "NaN"
            elif text.startswith("Duration:"):
                duration = div.find("span").text.strip() if span else "NaN"
            elif text.startswith("Status:"):
                status = div.find("span").text.strip() if span else "NaN"
            elif text.startswith("MAL:") or text.startswith("Scores:"):
                if span:
                    contents = span.contents
                    if len(contents) > 0:
                        rating = contents[0].strip() if isinstance(contents[0], str) else contents[0].text.strip()
                    if len(contents) > 1:
                        votes_text = contents[1].strip() if isinstance(contents[1], str) else contents[1].text.strip()
                        # Extract number only from votes string, e.g. 'by 13,298 users' -> 13298
                        votes_match = re.search(r'(\d[\d,]*)', votes_text)
                        votes = votes_match.group(1).replace(',', '') if votes_match else "NaN"
            elif text.startswith("Studios:"):
                studios = [a.text.strip() for a in span.find_all("a")] if span else "NaN"
            elif text.startswith("Producers:"):
                producers = [a.text.strip() for a in span.find_all("a")] if span else "NaN"
        
        genres_list.append(genres)
        country_list.append(country)
        premiered_list.append(premiered)
        date_aired_list.append(date_aired)
        broadcast_list.append(broadcast)
        duration_list.append(duration)
        status_list.append(status)
        rating_list.append(rating)
        votes_list.append(votes)
        studios_list.append(studios)
        producers_list.append(producers)
        
    df_demo = pd.DataFrame({
        "English Title": en_titles,
        "Japanese Title": jp_titles,
        "eng sub": sub_list,
        "eng dub": dub_list,
        "episode": ep_list,
        "type": type_list,
        "18+": tag_list,
        "URL": url_list,
        "Genres": genres_list,
        "Country": country_list,
        "premiered": premiered_list,
        "Release Date": date_aired_list,
        "Broadcast": broadcast_list,
        "EP1 duration": duration_list,
        "Status": status_list,
        "Rating": rating_list,
        "Votes": votes_list,
        "Studio": studios_list,
        "Producer": producers_list
    })
    
    df_list.append(df_demo)


                                                                                                                       1.52s/it]
                                                                                                                       1.52s/it]
                                                                                                                       1.59s/it]
                                                                                                                       1.69s/it]
                                                                                                                       1.49s/it]
                                                                                                                       1.54s/it]
                                                                                                                       1.68s/it]
                                                                                                

In [15]:
df = pd.concat(df_list, ignore_index = True)

In [16]:
df

,English Title,Japanese Title,eng sub,eng dub,episode,type,18+,URL,Genres,Country,premiered,Release Date,Broadcast,EP1 duration,Status,Rating,Votes,Studio,Producer
0,Slam Dunk: National Domination! Sakuragi Hanam...,SLAM DUNK: Zenkoku Seiha Da! Sakuragi Hanamichi,1,NaN,NaN,MOVIE,False,https://animekai.bz/watch/slam-dunk-national-d...,"[Comedy, School, Sports, Shounen, Drama, Slice...",Japan,Summer 1994,"Aug 20, 1994",,45 min,Completed,7.5,9584,[Toei Animation],
1,Slam Dunk: Shohoku Maximum Crisis! Burn Sakura...,SLAM DUNK: Shouhoku Saidai no Kiki! Moero Saku...,1,NaN,NaN,MOVIE,False,https://animekai.bz/watch/slam-dunk-shohoku-ma...,"[Comedy, School, Sports, Shounen, Drama, Slice...",Japan,Winter 1995,"Mar 04, 1995",,40 min,Completed,7.52,9421,[Toei Animation],
2,Slay The Gods,Zhan Shen: Fanchen Shen Yu,15,NaN,15.0,ONA,False,https://animekai.bz/watch/slay-the-gods-614v,"[Fantasy, Action, Psychological, Suspense]",China,Summer 2024,"Jul 31, 2024",,25 min,Completed,8.09,1380,[Shenman Entertainment],[Tencent Video]
3,Slayers,Slayers,26,26.0,26.0,TV,False,https://animekai.bz/watch/the-slayers-omnv,"[Adventure, Comedy, Fantasy, Action]",Japan,Spring 1995,"Apr 07, 1995",Fridays at 18:30 JST,22 min,Completed,7.72,64386,[E&G Films],"[J.C.Staff, TV Tokyo, TV Tokyo Music, SoftX]"
4,Slayers Excellent,Slayers Excellent,3,3.0,3.0,OVA,False,https://animekai.bz/watch/slayers-excellent-8ygk,"[Adventure, Comedy, Fantasy, Supernatural]",Japan,Fall 1998,"Oct 25, 1998",,29 min,Completed,7.28,10702,[J.C.Staff],[Bandai Visual]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,Zombie-Loan,Zombie-Loan,11,NaN,11.0,TV,False,https://animekai.bz/watch/zombie-loan-g5m7,"[Shounen, Action, Supernatural, Horror]",Japan,Summer 2007,"Jul 04, 2007",Wednesdays at 02:40 JST,23 min,Completed,6.81,65632,[XEBEC M2],"[Nihon Ad Systems, Xebec]"
2816,Zombie-Loan,Zombie-Loan Specials,2,NaN,2.0,SPECIAL,False,https://animekai.bz/watch/zombie-loan2-v0kk,"[Comedy, Shounen, Action, Supernatural, Horror]",Japan,Spring 2008,"Apr 25, 2008",,23 min,Completed,6.97,15413,[XEBEC M2],"[Nihon Ad Systems, Xebec]"
2817,Zombieland Saga Movie: Yumeginga Paradise,Zombieland Saga: Yume Ginga Paradise,Preview,NaN,NaN,MOVIE,False,https://animekai.bz/watch/zombie-land-saga-mov...,"[Comedy, Music, Supernatural, Parody]",Japan,Fall 2025,"Oct 24, 2025",,?,Info,10,1,[MAPPA],[Avex Pictures]
2818,Zone of the Enders: Dolores,"Z.O.E Dolores,i",26,26.0,26.0,TV,False,https://animekai.bz/watch/zoe-dolores-i-njnk,"[Adventure, Comedy, Sci-Fi, Action, Military, ...",Japan,Spring 2001,"Apr 07, 2001",Saturdays at 02:30 JST,24 min,Completed,6.92,4545,[Sunrise],"[AT-X, VAP, TV Tokyo Music]"


In [17]:
df["eng dub"] = pd.to_numeric(df["eng dub"], errors="coerce", downcast="integer").astype("Int64")
df["episode"] = pd.to_numeric(df["episode"], errors="coerce", downcast="integer").astype("Int64")
df["Release Date"] = pd.to_datetime(df["Release Date"], errors="coerce")
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce").astype("float16")

In [18]:
df = df.replace({
    "": "NaN",
    "?": "NaN",
    "by ? users": "NaN",
    "[]": "NaN"
})

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2820 entries, 0 to 2819
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   English Title   2820 non-null   object        
 1   Japanese Title  2820 non-null   object        
 2   eng sub         2820 non-null   object        
 3   eng dub         973 non-null    Int64         
 4   episode         2241 non-null   Int64         
 5   type            2820 non-null   object        
 6   18+             2820 non-null   bool          
 7   URL             2820 non-null   object        
 8   Genres          2820 non-null   object        
 9   Country         2820 non-null   object        
 10  premiered       2820 non-null   object        
 11  Release Date    2629 non-null   datetime64[ns]
 12  Broadcast       2820 non-null   object        
 13  EP1 duration    2820 non-null   object        
 14  Status          2820 non-null   object        
 15  Rati

In [20]:
# df.reset_index(drop=True).to_csv("all_anime_100.csv", index=False)

In [21]:
old_df = pd.read_csv("all_anime_300.csv", index_col=0)
old_df = pd.concat([old_df, df], ignore_index=True)
old_df

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,English Title,Japanese Title,eng sub,eng dub,episode,type,...,Country,premiered,Release Date,Broadcast,EP1 duration,Status,Rating,Votes,Studio,Producer
0,0.0,0.0,0.0,0.0,_Summer,_summer,2,<NA>,2.0,OVA,...,Japan,Fall 2006,2006-10-27,NaN,24 min,Completed,5.560000,5614.0,['Rikuentai'],NaN
1,1.0,1.0,1.0,1.0,_Summer Specials,_summer Specials,2,<NA>,2.0,SPECIAL,...,Japan,Fall 2006,2006-10-27,NaN,4 min,Completed,5.300000,1710.0,['Rikuentai'],['Rikuentai']
2,2.0,2.0,2.0,2.0,.hack//G.U. Returner,.hack//G.U. Returner,1,<NA>,1.0,OVA,...,Japan,Winter 2007,2007-01-18,NaN,24 min,Completed,6.650000,9714.0,['Bee Train'],"['Bandai Visual', 'CyberConnect2']"
3,3.0,3.0,3.0,3.0,.hack//G.U. Trilogy,.hack//G.U. Trilogy,1,<NA>,<NA>,MOVIE,...,Japan,Fall 2007,2007-12-22,NaN,93 min,Completed,7.060000,15307.0,['CyberConnect2'],['Bandai Visual']
4,4.0,4.0,4.0,4.0,.hack//Gift,.hack//GIFT,1,<NA>,1.0,OVA,...,Japan,Fall 2003,2003-11-16,NaN,26 min,Completed,6.090000,9975.0,['Bee Train'],['CyberConnect2']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11845,NaN,NaN,NaN,NaN,Zombie-Loan,Zombie-Loan,11,<NA>,11.0,TV,...,Japan,Summer 2007,2007-07-04 00:00:00,Wednesdays at 02:40 JST,23 min,Completed,6.808594,65632,[XEBEC M2],"[Nihon Ad Systems, Xebec]"
11846,NaN,NaN,NaN,NaN,Zombie-Loan,Zombie-Loan Specials,2,<NA>,2.0,SPECIAL,...,Japan,Spring 2008,2008-04-25 00:00:00,NaN,23 min,Completed,6.968750,15413,[XEBEC M2],"[Nihon Ad Systems, Xebec]"
11847,NaN,NaN,NaN,NaN,Zombieland Saga Movie: Yumeginga Paradise,Zombieland Saga: Yume Ginga Paradise,Preview,<NA>,<NA>,MOVIE,...,Japan,Fall 2025,2025-10-24 00:00:00,NaN,NaN,Info,10.000000,1,[MAPPA],[Avex Pictures]
11848,NaN,NaN,NaN,NaN,Zone of the Enders: Dolores,"Z.O.E Dolores,i",26,26.0,26.0,TV,...,Japan,Spring 2001,2001-04-07 00:00:00,Saturdays at 02:30 JST,24 min,Completed,6.921875,4545,[Sunrise],"[AT-X, VAP, TV Tokyo Music]"


In [22]:
old_df.to_csv("all_anime_400.csv", index=False)